In [145]:
import tensorflow as tf
import keras
import pandas as pd
import nltk
import numpy as np

In [118]:
#WordVectors

In [4]:
import gensim
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [6]:
train_DF=pd.read_csv('train.csv',index_col='id')

In [14]:
train_DF.reset_index(drop=True,inplace=True)
train_DF['question1'] = train_DF['question1'].astype(str)
train_DF['question2'] = train_DF['question2'].astype(str)

In [15]:
def word_count(a,b):
    return len(nltk.tokenize.word_tokenize(a)), len(nltk.tokenize.word_tokenize(b))

train_DF['word_count_in_question1'],train_DF['word_count_in_question2']= zip(*train_DF.apply(lambda row: 
                                                              word_count(row['question1'], row['question2']), axis=1))

In [84]:
import re
numbers=re.compile(r'(\d+\.\d+|\d+)')
def cleanText(text):
    return numbers.sub('N',text)

In [85]:
texts = []
questions1=[]
questions2=[]
duplicates=[]

for idx in range(train_DF.question1.shape[0]):
    text=train_DF.question1[idx]
    text=cleanText(text)
    texts.append(text)
    sentences = nltk.tokenize.sent_tokenize(text)
    questions1.append(sentences)
    
    text=train_DF.question2[idx]
    text=cleanText(text)
    texts.append(text)
    sentences = nltk.tokenize.sent_tokenize(text)
    questions2.append(sentences)
    
    duplicates.append(train_DF.is_duplicate[idx])

In [86]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 90278 unique tokens.


In [88]:
vocab_dim = 300 # dimensionality of your word vectors
n_symbols = len(word_index) + 1 # adding 1 to account for 0th index (for masking)
embedding_weights = np.random.random((n_symbols+1,vocab_dim))
for word,index in word_index.items():
    if word in model.wv.vocab:
        embedding_weights[index,:] = model.word_vec(word)       

In [109]:
MAX_SENT_LENGTH = 40
MAX_SENTS = 5

In [112]:
data_question1 = np.zeros((len(questions1), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
for i, sentences in enumerate(questions1):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTS:
            wordTokens = tf.keras.preprocessing.text.text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH:
                    data_question1[i,j,k] = tokenizer.word_index[word]
                    k=k+1

In [114]:
data_question2 = np.zeros((len(questions2), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
for i, sentences in enumerate(questions2):
    for j, sent in enumerate(sentences):
        if j< MAX_SENTS:
            wordTokens = tf.keras.preprocessing.text.text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH:
                    data_question2[i,j,k] = tokenizer.word_index[word]
                    k=k+1

In [155]:
embedding_layer=tf.keras.layers.Embedding(input_dim=len(word_index)+1,output_dim=vocab_dim,weights=embedding_weights,
                          input_length=MAX_SENT_LENGTH)

In [156]:
data_200=data_question1[:200]
duplicates_200=duplicates[:200]

In [157]:
sentence_input = tf.keras.layers.Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = tf.keras.layers.Bidirectional(LSTM(300))(embedded_sequences)
sentEncoder = tf.keras.models.Model(sentence_input, l_lstm)
sentEncoder.layers[1].trainable=False

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = tf.keras.layers.TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = tf.keras.layers.Bidirectional(LSTM(300))(review_encoder)
preds = tf.keras.layers.Dense(2, activation='softmax')(l_lstm_sent)
LSTM_model = tf.keras.models.Model(review_input, preds)

LSTM_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - LSTM")
print(LSTM_model.summary())

ValueError: You called `set_weights(weights)` on layer "embedding_6" with a  weight list of length 90280, but the layer was expecting 1 weights. Provided weights: [[ 0.56354204  0.92325401  0.05623875 ...,  0.1657...

In [158]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

In [159]:
embedding_layer=Embedding(input_dim=len(word_index)+1,output_dim=vocab_dim,weights=embedding_weights,
                          input_length=MAX_SENT_LENGTH)

In [160]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(300))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(300))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
LSTM_model = Model(review_input, preds)

LSTM_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - LSTM")
print(LSTM_model.summary())

ValueError: You called `set_weights(weights)` on layer "embedding_2" with a  weight list of length 90280, but the layer was expecting 1 weights. Provided weights: [[ 0.56354204  0.92325401  0.05623875 ...,  0.1657...